# **Installing Libraries**

In [3]:
!pip install -q langchain torch transformers sentence-transformers faiss-cpu unstructured gradio langchain-community langchain-huggingface tiktoken

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
cud

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Loading Dataset**

In [5]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('/kaggle/input/tour-prep', show_progress = True)
data = loader.load()

100%|██████████| 3/3 [00:09<00:00,  3.19s/it]


# **Split Documents into Chunks**

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    is_separator_regex=True
)
documents = text_splitter.split_documents(data)
len(documents)

427

In [7]:
# from langchain.text_splitter import TokenTextSplitter

# text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)
# docs = text_splitter.split_documents(data)
# len(docs)

**Print a random document chunk**

In [8]:
import random

print(random.choice(documents).page_content)

page parchment manuscript at bete gebrael constituting the third group and setting lonely bete giyorgis is architecturally an elegant sophisticated and most striking church this freely standing monolithic church symbolizes noahs ark owing to these sorts bete giyorgis is getting the standard of national symbol the church is entered through a tunnel that again leads to a trench in the courtyard there is a holy water where it flows from is not easily demarcated in the grave holes around the detached walls in the courtyard there are mummies that are easily seen by visitors in side the church along with big parchment manuscripts and crosses there is a large wooden box which was unusually locked by wooden nuts in which saint lalibela used to put the pristine carving tools the presence of this box here signifies that bete giyorgis is the lastly built church a tourist guide to the amhara region of ethiopia 81 to visit all these rockhewn churches meaningfully a couple of days are required note


# **Embedding and Vector Store**

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-11 06:53:55.265496: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 06:53:55.265609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 06:53:55.405608: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(documents, embeddings)

**Test Similarity Search**

In [11]:
question = "Tell me about Diredawa and its Surrounds"
search_docs = db.similarity_search(question)
print(f"Sample Search Result: {search_docs[0].page_content}")

Sample Search Result: settlements that flourished around the railway station and the other around the old market these two clearly defined centers are very much part of the character of the town today the open markets around 50 university of gondar cssh department of history and heritage management 2020 kefira are colorful attract locals from this town and its surrounds the railway town has bright jacarandas and flamboyant line the wide avenues the chemin de fer or railway station is dire dawas most attractive feature although the original steam engines have long since been replaced by diesels the trains running between dire dawa and addis ababa still have a wild pioneering atmosphere lega oda is found only 32 kilometers from dire dawa there are 2500 year old rock paintings in lega oda cave this rarely visited site has now become much more accessible by road and provides a fascinating glimpse of the ancient people who lived in and painted their cave with oxen and wild animals awash yan

**Retrieving Similar Chunks and Printing as a Dataframe**

In [12]:
import pandas as pd

query = "Attractions in and around gonder"

similar_chunks = db.similarity_search_with_relevance_scores(query, k=3)

retrieved_text = [chunk[0].page_content for chunk in similar_chunks]
relevance_score = [chunk[1] for chunk in similar_chunks]


retrieved_chunks = pd.DataFrame(
    list(zip(retrieved_text, relevance_score)),
    columns=["Retrieved Chunks", "Relevance Score"],
)
with pd.option_context("display.max_colwidth", None):
    display(retrieved_chunks)

,Retrieved Chunks,Relevance Score
0,use to recover from different diseases which drips from the roof church in the second section of the church kidist the church boasts various treasures including crosses crowns gold painted drum an illuminated bible attractions in and around gondar rd gondar was the 3 capital city in ethiopia next to aksum and lalibela respectively it was founded by emperor fasiledes in 1636 and become capital city for about 200 years gondar has been called as africas camelot churches and castles are the main attractions of the town some of the main attractions of gondar include royal enclosure gondars royal enclosure covers an area of 75000 sq m and was registered by unesco in 1979 the enclosure is surrounded by the high stone walls and connected by a series of tunnels and raised walkways there are six graceful castles in the palace compound th th built by the successive gondarian kings of the 17 and 18 c 1 fasiles castle is the oldest largest and the most impressive of the castles it stands two,0.356413
1,2020 circuit which served as ethiopias capital for almost 300 years the modern city of gondar is popular as a tourist attraction for its many picturesque ruins in the royal enclosure from which the emperors once reigned the most famous buildings in the city lie in the royal enclosure which include fasilades castle it is an old castle built in 17th century for the ethiopian emperor fasilides the other palaces and structures are iyasus palace dawits hall a banqueting hall stables mentewabs castle a chancellery library and three churches the royal enclosure lye within a high stone walled fortification with an area of 70000msq and 12 entrances inside the city the other attraction is,0.315752
2,fasilades bath during the heyday of gondar the bath had been used for sports and religious rites on the day of epiphany presently it is home to an annual timket ceremony where the water is blessed and then opened for bathing qusquam complex built by empress mentewab the eighteenth century ras mikael sehuls palace and the debre berhan selassie church are the other magnificent tourist attractions in gondar city to the north of gondar traditionally the preserve of hardened trekkers and hikers lie the amazingly scenic simien mountain national park home to the countrys main concentrations of the endemic gelada baboon walia ibex and ethiopian wolves the region includes many summits above 4000 meters and culminates in the highest point in ethiopia ras dashen which at 4543 meters is also africas fourth highest mountain iii axum a common third stop on the historic circuit is the ancient capital of axum which lies at the heart of the former axumite empire axum is best known today for the giant,0.274654


# **Load the LLM**

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = model.to(device)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# **Creating a Pipeline for Text Generation**

In [14]:
from transformers import pipeline

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    trust_remote_code=True,
    device_map="auto",
    max_length=512,
    do_sample=True,
    repetition_penalty=1.6,
    temperature=0.7,
    top_k=20,
    pad_token_id=tokenizer.eos_token_id
)

In [15]:
from langchain import LLMChain
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=text_generator)

# **Define the prompt template**

In [16]:
# from langchain_core.prompts.few_shot import FewShotPromptTemplate
# from langchain_core.prompts.prompt import PromptTemplate

# examples = [
#     {
#         "question": "Background of Ethiopian Commercial farm in 2017/18",
#         "answer": """
# In 2017/18 (E.C), which corresponds to 2015/16 (G.C), the agricultural sector in Ethiopia remained a cornerstone of the country's economy. Agriculture dominated the Ethiopian economy, significantly contributing to the Gross Domestic Product (GDP), employment, and foreign exchange earnings. This sector is essential for the livelihood of the majority of Ethiopians, who depend on it for food and income.

# Key Points:

# Economic Importance:

# Agriculture is crucial for Ethiopia's economic stability and growth. It generates a significant portion of the GDP and provides employment to a large segment of the population.
# The sector is pivotal for foreign exchange earnings, underpinning the country's economic resilience.
# Government and Stakeholder Involvement:

# The Ethiopian government, alongside various stakeholders including farmers, is actively involved in efforts to boost agricultural production and productivity.
# These efforts are aimed at ensuring food security and economic development by mitigating the challenges posed by the precarious nature of farming.
# Production and Challenges:

# Agriculture in Ethiopia is highly dependent on natural conditions, making it vulnerable to environmental changes and climate variability.
# Despite these challenges, farmers strive to increase production to meet both subsistence needs and market demands.
# Commercial Farms (LSF):

# Large Scale Farms (Commercial Farms) play a significant role in the agricultural landscape. They are involved in the production of various crops and livestock, contributing to the overall agricultural output.
# These farms employ modern agricultural practices, including the use of fertilizers like NPS (Nitrogen, Phosphorus, and Sulfur), to enhance productivity.
# Regional Distribution:

# Different regions in Ethiopia exhibit varying levels of agricultural activity. For instance, the SNNPR region shows high engagement in farming activities.
# Rural areas have a higher percentage of households involved in both crop and livestock farming compared to urban areas.
# Statistics:

# In the SNNPR region, 97.3% of households were engaged in farming activities, highlighting the region's agricultural focus.
# The national average for households involved in any form of farming (crop or livestock) stood at 91.1%.
# """,
#     },
#     {
#         "question": "One year Non-Food Expenditures of January-March 2012",
#         "answer": """
# Monthly Non-Food Expenditures:

# The table indicates the percentage of households purchasing these items and the mean expenditure in Birr for rural, small town, and combined areas:
# Laundry Soap: Purchased by 89.7% of households with a mean expenditure of 18 Birr.
# Matches: Purchased by 80.5% of households with a mean expenditure of 3 Birr.
# Batteries: Purchased by 59.9% of households with a mean expenditure of 10 Birr.
# Kerosene: Purchased by 56.5% of households with a mean expenditure of 12 Birr.
# Hand Soap: Purchased by 37.3% of households with a mean expenditure of 3 Birr.
# Other Personal Care Goods: Purchased by 34.8% of households with a mean expenditure of 3 Birr.
# Candles (tua'af), Incense: Purchased by 24.9% of households with a mean expenditure of 1 Birr.
# Transport: Purchased by 23.9% of households with a mean expenditure of 14 Birr.
# Cigarettes, Tobacco, Suret, Gaya: Purchased by 8.1% of households with a mean expenditure of 3 Birr.
# Firewood: Purchased by 6.3% of households with a mean expenditure of 3 Birr.
# Charcoal: Purchased by 4.1% of households with a mean expenditure of 1 Birr.
# Annual Non-Food Expenditures:

# The table for one-year expenditures includes both durable and non-durable goods:
# The expenditure covers items like clothing, equipment, furniture, taxes, levies, donations, and ceremonial expenses.
# The table details expenditures for the past 12 months, reflecting the average household spending on these items.
# Key Points:

# Frequency of Purchases:

# Certain items such as laundry soap, matches, and batteries were commonly purchased by the majority of households on a monthly basis.
# Items like candles, personal care goods, and transport services were less frequently purchased but still significant.
# Expenditure Variation:

# There is a notable variation in expenditure between rural and small town areas, with certain items being more commonly purchased in small towns.
# Transport services had a higher mean expenditure compared to other items, indicating its importance as a non-food expenditure.
# Durables and Non-Durables:

# The annual expenditure also includes durable goods and services, highlighting the broader scope of household spending beyond immediate consumables.
# Items such as clothing and furniture, along with expenses for taxes and ceremonies, reflect significant yearly financial commitments.
# In summary, the non-food expenditures for January-March 2012 illustrate the diverse spending patterns of Ethiopian households, with a focus on frequently purchased items and significant annual expenses for durable goods and services.
# """,
#     },
    
#         {
#         "question": "Give me the summary of Estimated Numbers of Sheep and Goats by Sex, Age, Breed, and Purpose, Ethiopia - 2015/16",
#         "answer": """
# Summary of Estimated Numbers of Sheep and Goats by Sex, Age, Breed, and Purpose, Ethiopia (2015/16)
# Overall Holdings:

# Total Holdings: 16,529,949
# Holdings with No Sheep/Goats:
# Sheep: 10,833,489 (65.54%)
# Goats: 12,011,096 (72.66%)
# Holdings with 1-4 Head:
# Sheep: 3,503,252 (21.19%)
# Goats: 2,597,587 (15.71%)
# Holdings with 5-9 Head:
# Sheep: 1,534,364 (9.28%)
# Goats: 1,183,988 (7.16%)
# Holdings with 10-49 Head:
# Sheep: 643,936 (3.9%)
# Goats: 698,710 (4.23%)
# Holdings with 50-99 Head:
# Sheep: 14,122 (0.09%)
# Goats: 29,160 (0.18%)
# Regional Estimates:

# Gambella:
# Sheep: 192
# Goats: 103
# Total Head of Livestock: 5,333 (sheep and goats combined)
# Harari:
# Sheep: 8,736
# Goats: 950
# Total Head of Livestock: 10,087 (sheep and goats combined)
# Dire Dawa:
# Sheep: 13,129
# Goats: 1,262
# Total Head of Livestock: 14,391 (sheep and goats combined)
# Purpose and Breed Information:

# The survey data includes estimates categorized by sex, age, and breed of sheep and goats.
# Data is collected to understand the purpose of the livestock, which may include breeding, meat production, or other uses.
# Additional Information:

# The majority of sheep and goat holdings consist of smaller herds, with large percentages having no sheep or goats at all.
# The detailed breakdown of regional estimates provides insight into livestock distribution across various areas of Ethiopia.
# The results are preliminary and subject to change based on further research into pasture use patterns and additional data collection.
# """
#  },
# ]

# example_prompt = PromptTemplate(
#     input_variables=["question", "answer"], template="Question: {question}\n\nAnswer: {answer}"
# )

In [17]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "Churches outside the town of Lalibela",
        "answer": """
Lalibela, a town in the Amhara Region of northern Ethiopia, is renowned for its monolithic rock-hewn churches, which are often considered among the wonders of the world. While the churches within Lalibela itself are the most famous, there are also significant rock-hewn churches outside the town that contribute to the region's historical and religious importance. Here are a few notable examples:

1. Yemrehana Krestos Church
Location: About 12 miles northeast of Lalibela.
Historical Significance: Built during the reign of King Yemrehana Krestos in the 11th or 12th century, predating the churches of Lalibela.
Architecture: The church is constructed within a large natural cave and features an intricate design combining Aksumite, early Christian, and traditional Ethiopian architectural styles. It is made of wood and stone, with beautifully carved and painted ceilings.
Features: The church also houses a mummified collection of pilgrims who sought refuge there.
2. Asheton Maryam Monastery
Location: Situated on Mount Abuna Yosef, about 6 miles from Lalibela.
Historical Significance: Believed to have been founded by King Lalibela himself.
Architecture: The church is carved out of rock and is perched on a mountain, offering panoramic views of the surrounding landscape.
Pilgrimage: It is a popular pilgrimage site, especially during religious festivals.
3. Genete Maryam Church
Location: Approximately 30 kilometers east of Lalibela.
Historical Significance: Dates back to the 13th century, during the reign of Emperor Yekuno Amlak.
Architecture: Unlike many of the other rock-hewn churches, Genete Maryam is a monolithic church built from a single piece of rock and features extensive frescoes and carvings.
Features: The church is noted for its colorful and well-preserved wall paintings.
4. Na’akuto La’ab Church
Location: A few kilometers outside Lalibela.
Historical Significance: Built by King Na’akuto La’ab, who succeeded King Lalibela.
Architecture: The church is set in a shallow cave and includes a series of pools fed by natural springs, considered holy water by pilgrims.
Features: The site is also known for its collection of religious artifacts, including ancient manuscripts and crosses.
5. Bilbala Giyorgis Church
Location: About 30 kilometers north of Lalibela.
Historical Significance: Dates to the 13th century and showcases the continuation of the rock-hewn church tradition in the region.
Architecture: The church is carved into a rock face and features intricate designs and iconography typical of Ethiopian Orthodox Christian art.
Importance of These Churches
These rock-hewn churches outside Lalibela are important for several reasons:

Historical Continuity: They illustrate the continuation and evolution of the rock-hewn church tradition in Ethiopia.
Architectural Diversity: Each church exhibits unique architectural styles and techniques, contributing to the rich architectural heritage of the region.
Pilgrimage and Tourism: They are significant pilgrimage sites for Ethiopian Orthodox Christians and attract tourists interested in religious history and architecture.
Together with the churches within Lalibela, these sites form a comprehensive and fascinating network of religious and historical monuments, reflecting the deep spiritual heritage and architectural ingenuity of medieval Ethiopia.
""",
    },
    {
        "question": "Do you Know about axum maryam tsion?",
        "answer": """
Axum (or Aksum) Maryam Tsion, also known as the Church of St. Mary of Zion, is one of the most significant religious sites in Ethiopia. Here is an overview of its importance and background:

Historical and Religious Significance
Location: The Church of St. Mary of Zion is located in the town of Axum, in the Tigray region of northern Ethiopia. Axum was the ancient capital of the Aksumite Empire, which was a major trading empire in the region.

Religious Importance: The church is one of the oldest Christian sites in Africa and is of paramount importance to Ethiopian Orthodox Christianity. It is traditionally believed to house the Ark of the Covenant, a sacred biblical artifact said to contain the stone tablets of the Ten Commandments.

Foundation: According to Ethiopian tradition, the original church was built during the 4th century AD by Emperor Ezana, who converted to Christianity and made it the state religion. This makes it one of the oldest Christian churches in the world.

Architecture: The original structure has undergone several reconstructions over the centuries. The current complex includes an old church built in the 17th century by Emperor Fasilides and a new church constructed in the 20th century by Emperor Haile Selassie.

The Ark of the Covenant
One of the most fascinating aspects of the Church of St. Mary of Zion is the belief that it houses the Ark of the Covenant. According to Ethiopian tradition:

The Ark was brought to Ethiopia by Menelik I, the son of the Queen of Sheba and King Solomon of Israel.
It has been kept in Ethiopia ever since, guarded by a succession of monks who dedicate their lives to this duty.
Pilgrimage Site
The church is a major pilgrimage site, especially during the annual festival of Tsion Mariam (St. Mary of Zion) celebrated on November 30th. Pilgrims from all over Ethiopia and beyond visit the church during this time.

Political and Cultural Influence
Throughout history, the Church of St. Mary of Zion has played a significant role in Ethiopian politics and culture:

It has been a coronation site for many Ethiopian emperors.
The church and its traditions have been central to the identity and continuity of Ethiopian Orthodox Christianity.
Modern Challenges
In recent years, the region has faced political instability, and the church has not been immune to the impacts of conflict, particularly due to the Tigray conflict that began in 2020. Preservation and protection of this historic and religious site remain important for both Ethiopians and the global community.
""",
    },
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n\nAnswer: {answer}"
)

# **Create the LLM Chain**

In [18]:
llm_chain = LLMChain(
    llm=llm,
    prompt=example_prompt,
)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


# **Create the StuffDocumentsChain**

In [19]:
from langchain.chains import RetrievalQA, StuffDocumentsChain

stuff_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="answer"
)

# **Combine into RetrievalQA Chain**

In [ ]:
qa = RetrievalQA(
    retriever=db.as_retriever(search_kwargs={'k': 1}),
    combine_documents_chain=stuff_docs_chain
)


# **Perform retrieval Q&A**

In [ ]:
from IPython.display import Markdown

qa_response = qa({"query": "Important HUman Fossils found in Lower Awash"})

Markdown(qa_response["result"])

In [22]:
# eval_questions = [
#     "What are Six Graceful castles around gonder?",
#     "Give me information about Dib Anbesa Hotel",
#     "Best food at bale national park",
# ]

# eval_answers = [
#     "1. Fasile’s castle: is the oldest, largest and the most impressive of the castles. It stands twostorey high and it is topped by four small domed towers. The palace is made up of roughly hewn brown basalt. It has dining room and reception area, prayer room with four windows which give the view of one of the many churches (1st floor), and bedroom (2nd floor).2. Palace of Yohannes I: which was once decorated with ivory and was built by the son and successor of Fasiledes; Yohannes. It is commonly known as the “Library of Yohannes”. 3. Palace of Iyasu I (Iyasu the Great): it was built by Iyasu I and looks like a saddle shaped castle. In the former times, it was well decorated. Visitors described it as „it is more beautiful than the others. But later on, it was damaged by the earthquake of 1704 and British bombingof 1940s.4. Castle of emperor Dawit III: used for religious, plentiful entertainments and lion houseAbyssinian lions. It is locally known as the “house of song”.5. Castle of emperor Bekaffa: it is a long V-shaped storied castle, located at the extreme north of the compound to the left side. It has two long towers; one of them was served as a banqueting and receptionist hall. 6. Mentuwab castle: two- storied structure now used as the house of souvenir products and public library. Empress Mentuwab (wife of emperor Bekaffa and mother of Iyasu II) built it. There are also other different attractions outside the royal enclosure.",
#     "Dib Anbesa HotelThis is the other sumptuous hotel, located between two squares. It isbuilt and furnished with beautiful woodcarvings. The ceilings and thedifferent columns are trimmed with colourful traditional paintings do have historical and artistic values.v Tel (251) 058 - 220 14 36v Fax (251) 058 - 220 18 18v P.O.Box - 982v Number of rooms: 60v Price range of the rooms: single, Birr 130 double, Birr 165 twin, Birr 185 suites, Birr 23014 A TOURIST GUIDE TO THE AMHARA REGION OF ETHIOPIAv Restaurant: there are both ala-cart and fixed menuswith delicious domestic and foreign dishes with avery reasonable price.v Bar: both local and imported beveragesOther Servicesl Secretarial servicel Telephone service in half of the roomsl Satellite TV in the bar and in the restaurantl Parking lotl Express laundryl Small gardenl Beauty saloonl Tailorl Air port -hotel transfer service l Tour operatorl Souvenir shopl Conference hall",
#     "Ambesha and Gomen Enjoy the only dish served in the small village of Rira located within the Harenna forest. Ambesha and gomen is a flatbread served with steamed kale, freshly harvested from nearby gardens. It’s so much better than it looks!"
# ]

# examples = [
#     {"query": q, "ground_truth": [eval_answers[i]]}
#     for i, q in enumerate(eval_questions)
# ]
